# 0. Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

# 1. 데이터 선정
## 1.1. 기계학습으로 해결하고자 하는 문제

창업에서 성공한 스타트업들은 어떤 조건을 갖추었을지 확인해보고 입력된 조건에 맞춰 스타트업의 창업 성공을 예측하고자 한다.

## 1.2. 데이터 선정

[Startup Data](https://www.kaggle.com/datasets/yanmaksi/big-startup-secsees-fail-dataset-from-crunchbase?resource=download)

해당 데이터의 정보를 확인해보면 아래와 같다.

In [ ]:
df = pd.read_csv("/content/big_startup_secsees_dataset.csv")

In [3]:
print(f"data length : {len(df)}")
print(f"data info : {df.info()}")
print(f"missing val : {df.isnull().sum()}")
df.head()

data length : 66368
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   permalink          66368 non-null  object
 1   name               66367 non-null  object
 2   homepage_url       61310 non-null  object
 3   category_list      63220 non-null  object
 4   funding_total_usd  66368 non-null  object
 5   status             66368 non-null  object
 6   country_code       59410 non-null  object
 7   state_code         57821 non-null  object
 8   region             58338 non-null  object
 9   city               58340 non-null  object
 10  funding_rounds     66368 non-null  int64 
 11  founded_at         51147 non-null  object
 12  first_funding_at   66344 non-null  object
 13  last_funding_at    66368 non-null  object
dtypes: int64(1), object(13)
memory usage: 7.1+ MB
data info : None
missing val : permalink                0
name      

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


In [4]:
def unique_count(df): #속성마다 유니크한 값의 개수를 반환해주는 함수
    unique_list = []
    for i in df.columns:
        uniq = df[i].unique()
        unique_list.append([i,len(uniq)]) #unique 값 개수
    return unique_list

In [5]:
unique_count(df)

[['permalink', 66368],
 ['name', 66103],
 ['homepage_url', 61192],
 ['category_list', 27297],
 ['funding_total_usd', 18896],
 ['status', 4],
 ['country_code', 138],
 ['state_code', 312],
 ['region', 1093],
 ['city', 5112],
 ['funding_rounds', 19],
 ['founded_at', 3979],
 ['first_funding_at', 4818],
 ['last_funding_at', 4518]]

### 데이터 제한 조건
* 최소 10개 이상의 feature → 성립

    위의 feature 중 전처리 과정에서 사용하기 어렵고 관계없는 feature는 'permalink', 'name', 'homepage_url'이다. 이 3가지를 제외하면 12가지의 feature를 가지고 있다.

* 최소 5,000개 이상의 instance → 성립

    총 6만개가량의 instance를 가지고 있다.

In [2]:
def clean_date(date_str):
    if pd.isna(date_str):
        return None
    try:
        date = pd.to_datetime(date_str)
        if date.year < 2000 or date.year > 2024:
            return None
        return date
    except:
        return None

def preprocess_startup_data(df):
    # 데이터 복사
    df = df.copy()

    # 날짜 컬럼 안전하게 변환
    date_columns = ['founded_at', 'first_funding_at', 'last_funding_at']
    for col in date_columns:
        df[col] = df[col].apply(clean_date)

    # 결측치 처리
    df['founded_at'] = df['founded_at'].fillna(df['first_funding_at'])

    # 기간 관련 feature 생성
    df['days_to_first_funding'] = (df['first_funding_at'] - df['founded_at']).dt.days.fillna(0)
    df['funding_duration'] = (df['last_funding_at'] - df['first_funding_at']).dt.days.fillna(0)
    df['company_age'] = (df['last_funding_at'] - df['founded_at']).dt.days.fillna(0)

    # funding_total_usd 전처리
    df['funding_total_usd'] = df['funding_total_usd'].str.replace('$', '').str.replace(',', '')
    df['funding_total_usd'] = pd.to_numeric(df['funding_total_usd'], errors='coerce')
    df['funding_total_usd'] = df['funding_total_usd'].fillna(0)
    df['funding_total_usd'] = np.log1p(df['funding_total_usd'])

    # 범주형 변수 전처리
    le = LabelEncoder()
    for col in ['country_code', 'state_code', 'region']:
        df[col] = df[col].fillna('Unknown')
        df[col] = le.fit_transform(df[col])

    # 도시 전처리
    df['city'] = df['city'].fillna('Unknown')
    top_cities = df['city'].value_counts().nlargest(100).index
    df['city'] = df['city'].apply(lambda x: x if x in top_cities else 'Others')
    df['city'] = le.fit_transform(df['city'])

    # 카테고리 전처리 부분만 이렇게 수정
    # 1. 먼저 모든 카테고리 데이터를 딕셔너리로 수집
    df['category_list'] = df['category_list'].fillna('Unknown')
    categories = set()
    for cats in df['category_list'].str.split(',', expand=False):
        if isinstance(cats, (list, pd.Series)):
            categories.update([cat.strip() for cat in cats])

    # 2. 빈 딕셔너리를 생성하여 모든 카테고리에 대한 결과를 저장
    category_dict = {}
    for category in categories:
        if category:  # 빈 문자열 제외
            category_dict[f'category_{category}'] = df['category_list'].str.contains(category, na=False).astype(int)

    # 3. 딕셔너리를 한 번에 데이터프레임으로 변환
    category_dummies = pd.DataFrame(category_dict)

    # 4. 원본 데이터프레임과 카테고리 데이터를 한 번에 결합
    df = pd.concat([df, category_dummies], axis=1)

    # 성공 여부 정의
    df['is_successful'] = 0
    df.loc[df['status'].isin(['ipo', 'acquired']), 'is_successful'] = 1
    df.loc[(df['status'] == 'operating') & (df['funding_rounds'] >= 3), 'is_successful'] = 1
    df.loc[(df['status'] == 'operating') & (df['company_age'] >= 1825), 'is_successful'] = 1

    # 불필요한 컬럼 제거
    columns_to_drop = ['permalink', 'name', 'homepage_url', 'category_list', 'status',
                      'founded_at', 'first_funding_at', 'last_funding_at']
    df = df.drop(columns=columns_to_drop)

    return df

# 실행
df = pd.read_csv("big_startup_secsees_dataset.csv")
preprocessed_df = preprocess_startup_data(df)

# 결과 확인
print("전처리된 데이터 shape:", preprocessed_df.shape)
print("결측치 수:", preprocessed_df.isnull().sum().sum())

# 저장
preprocessed_df.to_csv('preprocessed_startup_data.csv', index=False)

NameError: name 'pd' is not defined